In [6]:
import pandas as pd
import numpy as np 
from google.cloud import bigquery 

def bigq_todf(sql):
    cliente = bigquery.Client()
    df_resultado = cliente.query(sql).to_dataframe()
    print(df_resultado.head(1))
    return df_resultado

In [98]:
# querys para perfiles 
pp = '''
        SELECT DISTINCT * FROM `abi-martech-maz-col.MIDAS.MIDAS_DASHBOARD_GROUPED_PP`
        WHERE lower(PP) NOT LIKE '%beer%' 
        AND lower(PP) NOT LIKE '%coupon%'
    '''

demo = '''
select
td_id,
genero,
case when edad is null then 0
else
edad
end edad 
from(
select
a.td_id,
a.abi_gender,
a.abi_age edad,
b.abi_gender_predicted,
CASE WHEN a.abi_gender is null then b.abi_gender_predicted
ELSE a.abi_gender
END genero
from
`abi-martech-global.maz_col_cdp_inbound.L2_attributes` a
left join (
  select
  td_id,
  abi_gender_predicted
  from
  (select
key as td_id,
value as abi_gender_predicted
from
`abi-martech-global.maz_col_cdp_outbound.col_predicted_gender`
)
) b
on a.td_id = b.td_id
)
where genero is not null 
'''


In [99]:
pp=bigq_todf(pp)
# demo=bigq_todf(demo)

                                 td_id1                PP
0  953bda8e-fcfc-46bc-b224-d791bf74dbcf  Environment Care


In [100]:
pp.head(2)

,td_id1,PP
0,953bda8e-fcfc-46bc-b224-d791bf74dbcf,Environment Care
1,3cad6805-caf8-4394-b4a6-711e06034025,Environment Care


In [10]:
demo.shape

(6997800, 3)

In [16]:
demo.dtypes

td_id     object
genero    object
edad      object
Edad      object
dtype: object

In [18]:
demo['edad'] = pd.to_numeric(demo['edad'], errors='coerce')

In [19]:
demo['Edad'] = 'Sin edad'
demo.loc[(demo['edad'] >= 18) & (demo['edad'] <= 24), 'Edad'] = '18-24'
demo.loc[(demo['edad'] >= 25) & (demo['edad'] <= 34), 'Edad'] = '25-34'
demo.loc[(demo['edad'] >= 35) & (demo['edad'] <= 44), 'Edad'] = '35-44'
demo.loc[(demo['edad'] >= 45) & (demo['edad'] <= 60), 'Edad'] = '45-60'
demo.loc[(demo['edad'] >= 61), 'edad'] = '60+'
# Convertir la columna 'edad' a tipo de dato object
# demo['edad'] = demo['edad'].astype('object')


C:\Users\JuanSe\AppData\Local\Temp\ipykernel_20656\3375141193.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '60+' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  demo.loc[(demo['edad'] >= 61), 'edad'] = '60+'


In [20]:
demo.head(2)

,td_id,genero,edad,Edad
0,3d13fa94c21e6a4c806ae75891b7d9ff7cce1075c6d4d6...,Male,25.0,25-34
1,9b4ab3d2e6ae13e88b3d9c47b3c714ac338e8385732da3...,Male,0.0,Sin edad


In [21]:
# Leer csv de interacciones
interacciones = pd.read_csv('interacciones.csv')

In [22]:
interacciones.head()

,td_id1,nInteracciones,Dias_interaccion,abi_brand,CuadranteDias,CuadranteInteracciones,Cuadrante_Final
0,0b3f7960-b852-4c33-a650-96b9bf6338ef,2,274,Poker,Ice,Low,"3,1"
1,10e171d38361640e0ec4b439d6e6d6b2f4cbbc6746225f...,2,274,Poker,Ice,Low,"3,1"
2,313ba1e882f8e47c163ba2f676f05c719311b5e977d37f...,2,273,Poker,Ice,Low,"3,1"
3,502647d9-3185-47ed-bb20-b0aa4f3e8217,2,272,Poker,Ice,Low,"3,1"
4,2d0f89258a03529e6f5f0ca7f685e188f218c874d6f3c6...,2,272,Poker,Ice,Low,"3,1"


In [23]:
interacciones.shape

(165335, 7)

In [24]:
inter_demo=pd.merge(interacciones, demo, left_on='td_id1', right_on='td_id', how='left')

In [59]:
inter_demo[inter_demo['Cuadrante_Final']=='2,1']

,td_id1,nInteracciones,Dias_interaccion,abi_brand,CuadranteDias,CuadranteInteracciones,Cuadrante_Final,td_id,genero,edad,Edad
54992,19188a7c40555d4eb2654fe0183a9df3aa60e43380d8a8...,2,120,Poker,Midi,Low,"2,1",19188a7c40555d4eb2654fe0183a9df3aa60e43380d8a8...,Male,18.0,18-24
54996,871b721f1acbb909139e9c195fa2752c413a86cf312c30...,2,120,Poker,Midi,Low,"2,1",871b721f1acbb909139e9c195fa2752c413a86cf312c30...,Female,25.0,25-34
54998,53dd492df218d33525a7c6442bf921f4150f7c73c0c92b...,2,120,Poker,Midi,Low,"2,1",53dd492df218d33525a7c6442bf921f4150f7c73c0c92b...,Male,47.0,45-60
54999,30911fc0-25ec-4579-ba31-ca685600f0c8,2,120,Poker,Midi,Low,"2,1",30911fc0-25ec-4579-ba31-ca685600f0c8,Female,29.0,25-34
55005,7e05df160af591c0d7cb6d2441ee44a363932fcc06a08a...,2,120,Poker,Midi,Low,"2,1",7e05df160af591c0d7cb6d2441ee44a363932fcc06a08a...,Not Specified,30.0,25-34
...,...,...,...,...,...,...,...,...,...,...,...
116808,f1310e8aa1b0dfcd565b3a04fef4d01cd22f2b3e07aece...,2,83,Poker,Midi,Low,"2,1",f1310e8aa1b0dfcd565b3a04fef4d01cd22f2b3e07aece...,Male,27.0,25-34
116819,cd9570bce5a045843cb90de0c342b545df3e3c88f08cc0...,2,83,Poker,Midi,Low,"2,1",cd9570bce5a045843cb90de0c342b545df3e3c88f08cc0...,Female,25.0,25-34
116830,4247b72f0230aa9373131cb131cd77769ce5e4549ed145...,2,83,Poker,Midi,Low,"2,1",4247b72f0230aa9373131cb131cd77769ce5e4549ed145...,Female,50.0,45-60
116841,7cf669eb340ed2a204440c726976de3fa54ad0937f260a...,2,83,Poker,Midi,Low,"2,1",7cf669eb340ed2a204440c726976de3fa54ad0937f260a...,Female,26.0,25-34


In [26]:
keep = ['td_id1', 'Cuadrante_Final', 'genero', 'Edad']
subset_inter_demon = inter_demo[keep]

In [27]:
subset_inter_demon.head(2)

,td_id1,Cuadrante_Final,genero,Edad
0,0b3f7960-b852-4c33-a650-96b9bf6338ef,"3,1",Female,35-44
1,10e171d38361640e0ec4b439d6e6d6b2f4cbbc6746225f...,"3,1",Female,18-24


In [82]:
# top_genders = subset_inter_demon.groupby("genero")["td_id1"].nunique().nlargest(3)
# top_ages = subset_inter_demon.groupby("Edad")["td_id1"].nunique().nlargest(6)

top_genders = subset_inter_demon[subset_inter_demon.Cuadrante_Final=='3,3'].groupby("genero")["td_id1"].nunique().nlargest(3)
top_ages = subset_inter_demon[subset_inter_demon.Cuadrante_Final=='3,3'].groupby("Edad")["td_id1"].nunique().nlargest(6)

# top_genders = subset_inter_demon.groupby("genero")[["td_id1", 'Cuadrante_Final']].nunique().nlargest(3, columns=['td_id1', 'Cuadrante_Final'])
# top_ages = subset_inter_demon.groupby("Edad")[["td_id1", 'Cuadrante_Final']].nunique().nlargest(6, columns=['td_id1', 'Cuadrante_Final'])



In [83]:
top_genders

genero
Female           23694
Male              1467
Not Specified      299
Name: td_id1, dtype: int64

In [84]:
top_ages

Edad
25-34       8329
35-44       8066
45-60       5211
18-24       2710
Sin edad    1209
Name: td_id1, dtype: int64

In [101]:
inter_pp=pd.merge(interacciones, pp, on='td_id1', how='left')

In [102]:
inter_pp.head(2)

,td_id1,nInteracciones,Dias_interaccion,abi_brand,CuadranteDias,CuadranteInteracciones,Cuadrante_Final,PP
0,0b3f7960-b852-4c33-a650-96b9bf6338ef,2,274,Poker,Ice,Low,"3,1",NaN
1,10e171d38361640e0ec4b439d6e6d6b2f4cbbc6746225f...,2,274,Poker,Ice,Low,"3,1",NaN


In [103]:
inter_pp.PP.value_counts()

PP
Bars and Restaurants          7221
Futbol Lovers                 4542
Gaming                        3510
Travelers                     3227
Other Sports                  2956
Humor Fans                    2794
Foodies                       2348
Merch Lovers                  2031
Nature Fans                   1960
QR Scanners                   1935
Employment                    1630
Nabs Consumer                 1579
Better World                  1275
Environment Care              1275
Music Lover                   1270
Educación                      941
Fast Food                      775
Mundial                        764
Gourmet Foodies                615
Hinchas Selección Colombia     595
Home Consumption               584
Fashion & Beauty               546
Solteros                       398
Affordability                  389
Music Festivals                290
Heavy Consumer                 278
Copa America                   275
Hinchas Millonarios            223
Eco Friendly     

In [114]:
top_pps = inter_pp[inter_pp.Cuadrante_Final=='3,3'] .groupby("PP")["td_id1"].nunique().nlargest(5)
top_pps

PP
Gaming           71
Futbol Lovers    65
Other Sports     51
Travelers        42
Merch Lovers     38
Name: td_id1, dtype: int64

PP
Bars and Restaurants    7201
Futbol Lovers           4528
Gaming                  3502
Coupon Users            3387
Travelers               3223
Name: td_id1, dtype: int64

In [94]:
top_pps = inter_pp.groupby("PP")["td_id1"].nunique().nlargest(5)

,td_id1,nInteracciones,Dias_interaccion,abi_brand,CuadranteDias,CuadranteInteracciones,Cuadrante_Final,PP
0,0b3f7960-b852-4c33-a650-96b9bf6338ef,2,274,Poker,Ice,Low,"3,1",NaN
1,10e171d38361640e0ec4b439d6e6d6b2f4cbbc6746225f...,2,274,Poker,Ice,Low,"3,1",NaN
2,313ba1e882f8e47c163ba2f676f05c719311b5e977d37f...,2,273,Poker,Ice,Low,"3,1",NaN
3,502647d9-3185-47ed-bb20-b0aa4f3e8217,2,272,Poker,Ice,Low,"3,1",NaN
4,2d0f89258a03529e6f5f0ca7f685e188f218c874d6f3c6...,2,272,Poker,Ice,Low,"3,1",NaN


In [ ]:
top_pps = inter_pp.groupby("PP")["td_id1"].nunique().nlargest(5)

In [ ]:
def analyze_total_customers(df):
          
    # Create a dictionary to store the results for each cohort month
    results = {}
    
    customers_total = df['CUSTOMER_NK'].nunique()
    orders_total = df['ORDER_NK'].nunique()
    
    sort_order = ['Completo']
  
    for cluster in sorted(list(df[df['Cluster'].isin(sort_order)]['Cluster'].drop_duplicates()), key=lambda x: sort_order.index(x)):
        
        # Filter for customers who made a purchase in the cohort month or later
        retained_customers = df[df['Cluster'] == cluster]
        
        ids = retained_customers[['td_id', 'CUSTOMER_NK']].drop_duplicates().reset_index()
        
        frecuencia = retained_customers.groupby(['CUSTOMER_NK']).agg({'ORDER_NK' : 'nunique', 'AñoMes' : 'nunique'})
        frecuencia['Freq'] = frecuencia['ORDER_NK']/frecuencia['AñoMes']
        
        dates = retained_customers[['CUSTOMER_NK', 'ORDER_NK', 'ORDER_DATE']].drop_duplicates().sort_values(['CUSTOMER_NK', 'ORDER_DATE']).reset_index()
        dates['PREVIOUS_ORDER'] = dates.groupby(['CUSTOMER_NK'])['ORDER_DATE'].shift()
        dates = dates[-dates['PREVIOUS_ORDER'].isna()]
        dates['TBO'] = (dates['ORDER_DATE'] - dates['PREVIOUS_ORDER']).dt.days
        
        ARPU_mes = retained_customers.groupby(['CUSTOMER_NK']).agg({'NR' : 'sum', 'AñoMes' : 'nunique'})
        ARPU_mes['ARPU'] = ARPU_mes['NR']/ARPU_mes['AñoMes']
        
        Lts_mes = retained_customers.groupby(['CUSTOMER_NK']).agg({'TOTAL_HLS' : 'sum', 'AñoMes' : 'nunique'})
        Lts_mes['Lts'] = (Lts_mes['TOTAL_HLS']*100)/Lts_mes['AñoMes']
        
        units_order = retained_customers.groupby(['ORDER_NK']).agg({'Unidades' : 'sum'})
        units_order = units_order[units_order['Unidades']<=units_order['Unidades'].quantile(0.99)]
        
        # contact = retained_customers.groupby(['Contacto']).agg({'CUSTOMER_NK' : 'nunique'}).nlargest(columns='CUSTOMER_NK', n=2)
        
        pps = pd.merge(ids, pp, left_on='td_id', right_on='td_id1', how='inner')
        
        # Calculate summary statistics for the retained customers
        cohort_size = retained_customers["CUSTOMER_NK"].nunique()
        unique_months = retained_customers['AñoMes'].nunique()
        total_revenue = retained_customers["NR"].sum()
        ltrs = retained_customers['TOTAL_HLS'].sum()*100
        intensity_ltrs = ltrs/cohort_size
        intensity_ltrs_mes = Lts_mes['Lts'].mean()
        arpu = total_revenue/cohort_size
        arpu_mes = ARPU_mes['ARPU'].mean()
        total_orders = retained_customers["ORDER_NK"].nunique()
        aov = total_revenue / total_orders
        #frequency = total_orders / cohort_size
        frequency = frecuencia['Freq'].mean()
        rgb = retained_customers[retained_customers['RETORNABLE'] == 'Retornable']['ORDER_NK'].nunique()
        returnable = rgb/total_orders
        units = units_order['Unidades'].mean()
        
        porc_customers = cohort_size/customers_total
        porc_orders = total_orders/orders_total
        
        top_genders = retained_customers.groupby("genero")["CUSTOMER_NK"].nunique().nlargest(3)
        
        top_ages = retained_customers.groupby("Edad")["CUSTOMER_NK"].nunique().nlargest(6)
        
        # Get the top 3 brands bought by the retained customers in the current month
        top_brands = retained_customers.groupby("MARCA")["NR"].sum().nlargest(10)
        
        # Get the top 3 segments bought by the retained customers in the current month
        top_segments = retained_customers.groupby("SEGMENTO")["NR"].sum().nlargest(3)
        
        top_pps = pps.groupby("PP")["CUSTOMER_NK"].nunique().nlargest(5)
        
        # Get the top 3 SKUs bought by the retained customers in the current month
        top_skus = retained_customers.groupby("ITEM_NAME")["UNIT_NET_REVENUE_LOCAL"].sum().nlargest(5)
        
        top_promos = retained_customers.groupby("descuento")["ORDER_NK"].nunique().nlargest(3)
        
        top_weekday = retained_customers.groupby("DiaSemana")["ORDER_NK"].nunique().nlargest(5)
        
        top_hour = retained_customers.groupby("Hora")["ORDER_NK"].nunique().nlargest(5)
        
        top_nse = retained_customers.groupby("ESTRATO")["CUSTOMER_NK"].nunique().nlargest(6)
        
        top_payment = retained_customers.groupby("PAYMENT_TYPE")["ORDER_NK"].nunique().nlargest(4)
        
        top_devices = retained_customers.groupby("OS_NAME")["CUSTOMER_NK"].nunique().nlargest(3)
        
        top_discount = retained_customers.groupby("Rango descuento")['ORDER_NK'].nunique().nlargest(5)
        
        # Add the results to the dictionary
        results[cluster] = {
            "Cluster": cluster,
            #"purchase_month": month.strftime("%Y-%m"),
            "Customers": "{:,.0f}".format(cohort_size),
            "% Customers" : format(porc_customers, ".0%"),
            "Orders": "{:,.0f}".format(total_orders),
            "% Orders" : format(porc_orders, ".0%"),
            "ARPU": "${:,.0f}".format(arpu),
            "ARPU Mes": "${:,.0f}".format(arpu_mes),
            "Frequency": "{:.2f}".format(frequency),
            "TBO (Days)" : dates['TBO'].mean().round(),
            "Intensity Ltrs": "{:.0f}".format(intensity_ltrs),
            "Intensity Ltrs Mes": "{:,.0f}".format(intensity_ltrs_mes),
            "Units" : "{:,.2f}".format(units),
            "AOV": "${:,.0f}".format(aov),
            "Gender": ", ".join(top_genders.index.tolist()),
            "Gender %" : ", ".join(((top_genders/cohort_size)*100).round(2).astype(str) + '%'),
            "Age": ", ".join(top_ages.index.tolist()),
            "Age %" : ", ".join(((top_ages/cohort_size)*100).round(2).astype(str) + '%'),
            "Top 3 NSE Customers": ", ".join(top_nse.index.tolist()),
            "% NSE": ", ".join(((top_nse/cohort_size)*100).round(2).astype(str) + '%'),
            "Top 3 Brands Orders": ", ".join(top_brands.index.tolist()),
            "% Brands": ", ".join(((top_brands/total_revenue)*100).round(2).astype(str) + '%'),
            "Top PPs": ", ".join(top_pps.index.tolist()),
            "% PPs": ", ".join(((top_pps/cohort_size)*100).round(2).astype(str) + '%'),
            "Mix NR": ", ".join(top_segments.index.tolist()),
            "% Mix": ", ".join(((top_segments/total_revenue)*100).round(2).astype(str) + '%'),
            "Returnable" : format(returnable, ".0%"),
            "Payment methods Orders": ", ".join(top_payment.index.tolist()),
            "% Payment methods": ", ".join(((top_payment/total_orders)*100).round(2).astype(str) + '%'),
            "Discount range Orders": ", ".join(top_discount.index.tolist()),
            "% Discount range Orders": ", ".join(((top_discount/total_orders)*100).round(2).astype(str) + '%'),
            # "Contactability" : ", ".join(contact.index.astype(str).tolist()),
            # "% Contactability": ", ".join(((contact/cohort_size)*100).round(2).astype(str) + '%'),
            "Customer device": ", ".join(top_devices.index.tolist()),
            "% Customer device": ", ".join(((top_devices/cohort_size)*100).round(2).astype(str) + '%'),
            "top_promos" : ", ".join(top_promos.index.tolist()),
            "top_promos_share_orders": ", ".join(((top_promos/total_orders)*100).round(2).astype(str) + '%'),
            "top_weekday" : ", ".join(top_weekday.index.tolist()),
            "top_weekday_share_orders": ", ".join(((top_weekday/total_orders)*100).round(2).astype(str) + '%'),
            "top_hour" : ", ".join(top_hour.index.tolist()),
            "top_hour_share_orders": ", ".join(((top_hour/total_orders)*100).round(2).astype(str) + '%'),
            "top_3_skus": ", ".join(top_skus.index.tolist())
        }
               
    # Convert the dictionary to a DataFrame and return it
    df_results = pd.DataFrame.from_dict(results)
    
    return results